# Scrapper para Twitter

O script abaixo é um scrapper simples para twitter, ele pega em um intervalo de tempo determinado os tuites mais recentes especificados dentro de uma query especificada pelo usuário.

Acesso ao drive para salvamento dos arquivos necessários para a varredura e resultado final da busca em um arquivo no formato XLSX.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Importação dos módulos necessários para o funcionamento do script.

In [ ]:
import tweepy
import time
import datetime
import pandas as pd
from IPython.display import clear_output

Credenciais necessárias para acesso à API do Twitter.

In [ ]:
consumer_key = ""
consumer_secret = ""
access_token = ""
access_token_secret = ""

Iniciando a API do Twitter

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

Declaração de função que ajudará a manter a lista com os últimos tuites capturados em um tamanho aceitável

In [ ]:
def sizable_list(original_list):
    if len(original_list) >= 200:
        return original_list[-100:]
    if len(original_list) >= 100:
        return original_list[-50:]
    else:
        return original_list

**IMPORTANTE!**

Adicione entre aspas a query que você pretende buscar na variável `text_query` e quantas vezes o código vai buscar por novos tuites na variável `limit`*.

**OBS: Cada iteração foi configurada para rodar a cada 5 segundos, então tenha isso em mente para quando for configurar este parâmetro.*

Ex:


*   `limit = 2` representará 10 segundos de busca

*   `limit = 12` representará 1 minuto de busca

*   `limit = 600` representará 30 minutos de busca



In [ ]:
text_query = 'bolsonaro'
limit = 5

Cria a lista temporária de moniroramento.

In [ ]:
monitoring = pd.DataFrame({}, columns=['time', 'tweet_id', 'account', 'user_name', 'profile_location', 'tweet_text', 'tweet_url'], index=None)
monitoring.to_csv('monitoring.csv', index=False )

Parâmetros para o funcionamento da busca:


*   tweets = lista de tuites resgatado por busca
*   count = número de tuites resgatado por busca (máximo 100)
*   counter = contador de iterações
*   data_frame_size = garante que o tamanho do data frame será passado como um numeral mesmo se vazio.




In [ ]:
tweets = []
count = 100

counter = 0
data_frame_size = monitoring.index.max() if type(monitoring.index.max()) == type(0) else 0
tweet_list = sizable_list(monitoring['tweet_id'].tolist())

Script que faz toda a magia

In [ ]:
while counter < limit:

    clear_output(wait=True)
    tweets = []
    print('Running')
    try:
        # Pulling individual tweets from query
        for tweet in api.search(q=text_query, count=count, tweet_mode="extended", result_type="recent"):
            # Adding to list that contains all tweets
            # print(tweet)
            if not (tweet.full_text[0:2] == "RT") and not tweet.id in tweet_list:
                tweet_list.append(tweet.id)
                tweets.append([tweet.created_at, tweet.id, tweet.user.screen_name, tweet.user.name, tweet.user.location, tweet.full_text, f'http://www.twitter.com/{tweet.user.screen_name}/status/{tweet.id}'])

                data_frame_size += 1

    except BaseException as e:
        print('failed on_status,',str(e))

    # Export code
    tweets_df = pd.DataFrame(tweets, index=None)
    tweets_df.to_csv('monitoring.csv', mode='a', index=False, header=False)

    print(f'last tweet ID: {tweet_list[-1]}')
    print(f'tweets collected: {data_frame_size}')
    print(f'current iteration {counter+1}/{limit}')

    counter += 1
    time.sleep(5)

    if (data_frame_size >= 500000):
        print("Done")
        break

Running
last tweet ID: 1268632741853179905
tweets collected: 36
current iteration 5/5


Parte final do código onde é salvo a planilha.

In [ ]:
final = pd.read_csv('monitoring.csv', index_col=False)
final.to_excel(f'{text_query}_{datetime.datetime.now()}.xlsx', index=False )